In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as DataLoader
from torchvision import datasets, transforms
from tqdm.auto import tqdm
from torch.optim.lr_scheduler import ReduceLROnPlateau
import matplotlib.pyplot as plt


In [2]:
class CNN(nn.Module):
    def __init__(self, num_classes=10):
        super(CNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 3 * 3, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, num_classes)
        )
 
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x
 
print(f"Model has {sum(p.numel() for p in CNN().parameters())} parameters.")

        

Model has 246026 parameters.


In [3]:
class ModelTrainer:
    def __init__(self, model, train_loader, val_loader, criterion, optimiser, scheduler, device):
        self.model = model
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.criterion = criterion
        self.optimiser = optimiser
        self.scheduler = scheduler
        self.device = device
        self.history = {'train_loss': [], 'val_loss': [], 'val_acc': []}
 
        print(f"Using device: {self.device}")
 
    def train_epoch(self, epoch, num_epochs):
        self.model.train()
        train_loss = 0.0
        loop = tqdm(self.train_loader, desc=f"Epoch [{epoch+1}/{num_epochs}]")
 
        for inputs, targets in loop:
            inputs, targets = inputs.to(self.device), targets.to(self.device)
 
            self.optimiser.zero_grad()
            outputs = self.model(inputs)
            loss = self.criterion(outputs, targets)
            loss.backward()
            self.optimiser.step()
 
            train_loss += loss.item() * inputs.size(0)
            loop.set_postfix(loss=loss.item())
 
        return train_loss / len(self.train_loader.dataset)
 
    def validate(self):
        self.model.eval()
        val_loss = 0.0
        correct = 0
 
        with torch.no_grad():
            for inputs, targets in self.val_loader:
                inputs, targets = inputs.to(self.device), targets.to(self.device)
                outputs = self.model(inputs)
                loss = self.criterion(outputs, targets)
 
                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                correct += (predicted == targets).sum().item()
 
        val_loss /= len(self.val_loader.dataset)
        val_acc = correct / len(self.val_loader.dataset)
        return val_loss, val_acc
 
    def train(self, num_epochs):
        for epoch in range(num_epochs):
            train_loss = self.train_epoch(epoch, num_epochs)
            val_loss, val_acc = self.validate()
 
            self.history['train_loss'].append(train_loss)
            self.history['val_loss'].append(val_loss)
            self.history['val_acc'].append(val_acc)
 
            self.scheduler.step(val_loss)
 
            print(f"Epoch [{epoch+1}/{num_epochs}] - "
                  f"Train Loss: {train_loss:.4f}, "
                  f"Val Loss: {val_loss:.4f}, "
                  f"Val Acc: {val_acc:.4f}")
 
        print("Training complete.")
        return self.history
 
# %%
def plot_history(history):
    epochs = range(1, len(history['train_loss']) + 1)
 
    plt.figure(figsize=(12, 5))
 
    plt.subplot(1, 2, 1)
    plt.plot(epochs, history['train_loss'], label='Train Loss')
    plt.plot(epochs, history['val_loss'], label='Val Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Loss over Epochs')
    plt.legend()
 
    plt.subplot(1, 2, 2)
    plt.plot(epochs, history['val_acc'], label='Val Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Validation Accuracy over Epochs')
    plt.legend()
 
    plt.show()

In [4]:
BATCH_SIZE = 64
LEARNING_RATE = 0.001
NUM_EPOCHS = 10
 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
 
transforms = transforms.Compose([
    transforms.ToTensor(),
])
 
train_ds = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transforms)
val_ds = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transforms)
 
train_loader = DataLoader.DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader.DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)
 
model = CNN(num_classes=10).to(device)
cirterion = nn.CrossEntropyLoss()
optimiser = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = ReduceLROnPlateau(optimiser, mode='min', factor=0.1, patience=3)
 
trainer = ModelTrainer(model, train_loader, val_loader, cirterion, optimiser, scheduler, device)
 
history = trainer.train(NUM_EPOCHS)
 
final_loss, final_acc = trainer.validate()
print(f"Final Validation Loss: {final_loss:.4f}, Final Validation Accuracy: {final_acc:.4f}")
plot_history(history)

Using device: cuda


Epoch [1/10]:   0%|          | 0/938 [00:00<?, ?it/s]

RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x128 and 1152x128)